^C
^C


  Using cached yolo-0.3.2-py3-none-any.whl.metadata (1.0 kB)
  Using cached awscli-1.34.11-py3-none-any.whl.metadata (11 kB)
  Using cached boto3-1.35.11-py3-none-any.whl.metadata (6.6 kB)
  Using cached botocore-1.35.11-py3-none-any.whl.metadata (5.7 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached docker-3.4.0-py2.py3-none-any.whl.metadata (3.6 kB)
  Using cached keyring-8.7-py2.py3-none-any.whl.metadata (11 kB)
  Using cached keyrings.alt-5.0.2-py3-none-any.whl.metadata (3.6 kB)
INFO: pip is looking at multiple versions of docker to determine which version is compatible with other requirements. This could take a while.
  Using cached yolo-0.3.1.tar.gz (44 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/50.1 kB ? eta -:--:--
     ---------------------------------------- 50.1/50.1 kB 2.5 MB/s eta 0:00:00
  Using cached pyasn1-0.6.0-py2.py3-none-an


0: 640x448 6 persons, 241.1ms
Speed: 5.3ms preprocess, 241.1ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 448)
Elbow angle: 144.85 degrees
Knee angle: 146.85 degrees

0: 640x448 7 persons, 287.8ms
Speed: 4.0ms preprocess, 287.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 448)
Elbow angle: 148.59 degrees
Knee angle: 148.25 degrees

0: 640x448 6 persons, 270.2ms
Speed: 4.0ms preprocess, 270.2ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 448)
Elbow angle: 149.89 degrees
Knee angle: 148.89 degrees

0: 640x448 7 persons, 236.9ms
Speed: 4.0ms preprocess, 236.9ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 448)
Elbow angle: 151.73 degrees
Knee angle: 153.05 degrees

0: 640x448 7 persons, 225.9ms
Speed: 5.0ms preprocess, 225.9ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 448)
Elbow angle: 150.81 degrees
Knee angle: 155.33 degrees

0: 640x448 6 persons, 213.6ms
Speed: 4.0ms preprocess, 213.6ms inference, 3.1ms

In [11]:
from ultralytics import YOLO
import numpy as np
import cv2
import csv

def calculate_angle(point1, point2, point3): 
    a = np.array(point1)
    b = np.array(point2)
    c = np.array(point3)
    
    ab = a - b
    cb = c - b
    
    cosine_angle = np.dot(ab, cb) / (np.linalg.norm(ab) * np.linalg.norm(cb))
    angle = np.arccos(cosine_angle)
    
    return np.degrees(angle)

def is_outlier(current_point, previous_point, threshold=50):
    return np.linalg.norm(np.array(current_point) - np.array(previous_point)) > threshold

skeleton_connections = [
    (5, 7), (7, 9),   # 왼팔
    (6, 8), (8, 10),  # 오른팔
    (11, 13), (13, 15),  # 왼다리
    (12, 14), (14, 16),  # 오른다리
    (5, 6),  # 어깨 연결
    (11, 12),  # 엉덩이 연결
    (5, 11), (6, 12)  # 몸통 연결
]

model = YOLO('yolov8n-pose.pt')  
cap = cv2.VideoCapture('Steph1_test.mp4')  

csv_filename = 'keypoints_data.csv'
previous_keypoints = None  

with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    header = ['frame_number', 'keypoint', 'Elbow angle', 'Knee angle', 'Shoulder-Elbow angle', 'Elbow-Wrist angle']
    writer.writerow(header)

    frame_number = 1  
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)
        keypoints = results[0].keypoints.xyn.numpy()  
        boxes = results[0].boxes.xyxy.numpy()

        max_area = 0
        selected_person = None

        for person, box in zip(keypoints, boxes):
            x1, y1, x2, y2 = box
            area = (x2 - x1) * (y2 - y1)  

            if area > max_area:
                max_area = area
                selected_person = person

        if selected_person is not None:
            if previous_keypoints is not None:
                for i, keypoint in enumerate(selected_person):
                    if is_outlier(keypoint[:2] * [frame.shape[1], frame.shape[0]], previous_keypoints[i]):
                        selected_person[i] = previous_keypoints[i] / [frame.shape[1], frame.shape[0]]

            previous_keypoints = selected_person[:, :2] * [frame.shape[1], frame.shape[0]]

            if 1 <= frame_number <= 150:  # 1frame 부터 150frame 까지  (86부터 슛)
                left_shoulder = selected_person[5][:2] * [frame.shape[1], frame.shape[0]]
                left_elbow = selected_person[7][:2] * [frame.shape[1], frame.shape[0]]
                left_wrist = selected_person[9][:2] * [frame.shape[1], frame.shape[0]]
                elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

                left_hip = selected_person[11][:2] * [frame.shape[1], frame.shape[0]]
                left_knee = selected_person[13][:2] * [frame.shape[1], frame.shape[0]]
                left_ankle = selected_person[15][:2] * [frame.shape[1], frame.shape[0]]
                knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

                shoulder_elbow_angle = calculate_angle(left_hip, left_shoulder, left_elbow)

                # 추가된 부분: 어깨 - 팔꿈치 - 손목 각도 계산
                elbow_wrist_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

                print(f"Elbow angle: {elbow_angle:.2f} degrees")
                print(f"Knee angle: {knee_angle:.2f} degrees")
                print(f"Shoulder-Elbow angle: {shoulder_elbow_angle:.2f} degrees")
                print(f"El



0: 640x448 6 persons, 238.6ms
Speed: 8.7ms preprocess, 238.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 448)
Elbow angle: 144.85 degrees
Knee angle: 146.85 degrees
Shoulder-Elbow angle: 6.02 degrees

0: 640x448 7 persons, 257.7ms
Speed: 6.9ms preprocess, 257.7ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 448)
Elbow angle: 148.59 degrees
Knee angle: 148.25 degrees
Shoulder-Elbow angle: 4.85 degrees

0: 640x448 6 persons, 329.4ms
Speed: 5.4ms preprocess, 329.4ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 448)
Elbow angle: 149.89 degrees
Knee angle: 148.89 degrees
Shoulder-Elbow angle: 5.70 degrees

0: 640x448 7 persons, 216.7ms
Speed: 5.0ms preprocess, 216.7ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 448)
Elbow angle: 151.73 degrees
Knee angle: 153.05 degrees
Shoulder-Elbow angle: 7.38 degrees

0: 640x448 7 persons, 424.4ms
Speed: 10.0ms preprocess, 424.4ms inference, 3.6ms postprocess per image at shape (1, 3, 640,